In [1]:
import pandas as pd
import tushare as ts

In [3]:
pledge = pd.read_csv('../data/pledge/pledge_neg.csv', 
                     parse_dates=['start_date'])

In [4]:
pledge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27293 entries, 0 to 27292
Data columns (total 6 columns):
ts_code                27293 non-null object
start_date             27293 non-null datetime64[ns]
release_date           16054 non-null object
pledge_price           27293 non-null float64
forcast_close_line     27293 non-null float64
is_reach_close_line    27293 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(2), object(2)
memory usage: 1.1+ MB


In [5]:
pledge.head()

,ts_code,start_date,release_date,pledge_price,forcast_close_line,is_reach_close_line
0,000002.SZ,2018-12-25,2019-08-26,22.4751,15.73257,False
1,000002.SZ,2018-12-18,2019-07-22,24.3593,17.05151,False
2,000002.SZ,2019-06-11,NaN,26.7721,18.74047,False
3,000002.SZ,2018-09-26,2019-06-05,22.9654,16.07578,False
4,000002.SZ,2018-09-28,2019-06-05,23.0231,16.11617,False


In [6]:
code_list = set(pledge['ts_code'])

In [7]:
len(code_list)

2222

In [8]:
fund_dict = {
    code: pd.read_csv('../data/fund/company_fund_{}.csv'.format(code), 
                      index_col=0, 
                      parse_dates=['end_date']) 
    for code in code_list
}

In [9]:
for i in reversed(range(len(fund_dict['000002.SZ']))):
    print(i)

13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [10]:
fund_dict['000002.SZ']

,ts_code,ann_date,end_date,current_ratio,quick_ratio,cash_ratio,inv_turn,ar_turn,ca_turn,netprofit_margin,...,q_gr_yoy,q_gr_qoq,q_sales_yoy,q_sales_qoq,q_op_yoy,q_op_qoq,q_profit_yoy,q_profit_qoq,revenue,net_profit
0,000002.SZ,20190821,2019-06-30,1.1191,0.4387,0.3268,0.1153,80.7413,0.1074,13.8427,...,21.0203,88.0024,21.0203,88.0024,31.8907,273.2963,36.9291,401.5618,1.393201e+11,1.928568e+10
1,000002.SZ,20190430,2019-03-31,1.1594,0.4611,0.3498,0.0411,28.2571,0.0374,6.6273,...,56.9300,-60.2369,56.9300,-60.2369,98.0949,-83.5804,80.1607,-88.3312,1.393201e+11,NaN
2,000002.SZ,20190326,2018-12-31,1.1543,0.4856,0.3977,0.2760,197.2095,0.2574,16.5521,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,NaN
3,000002.SZ,20181026,2018-09-30,1.1612,0.4832,0.3543,0.1759,111.1831,0.1579,12.3836,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,4.927229e+10
4,000002.SZ,20180821,2018-06-30,1.1805,0.5220,0.4031,0.1124,66.0165,0.0980,12.7602,...,46.7143,143.7873,46.7143,143.7873,37.6351,460.6770,31.2987,559.9162,2.976793e+11,NaN
5,000002.SZ,20180426,2018-03-31,1.2104,0.4952,0.3828,0.0330,21.1921,0.0296,5.7728,...,65.8251,-75.4957,65.8251,-75.4957,90.5090,-90.1376,60.4339,-92.0530,1.760221e+11,1.352260e+10
6,000002.SZ,20180327,2017-12-31,1.2009,0.4950,0.3998,0.3005,138.4822,0.2794,15.3186,...,1.9236,166.0109,1.9236,166.0109,29.6575,328.7667,31.2536,370.0679,1.059745e+11,NaN
7,000002.SZ,20171027,2017-09-30,1.2102,0.4697,0.3140,0.1571,69.3443,0.1445,12.6528,...,11.9039,-7.6750,11.9039,-7.6750,19.4354,-41.9464,13.5359,-46.7393,3.082562e+10,NaN
8,000002.SZ,20170825,2017-06-30,1.2013,0.4673,0.3459,0.0976,37.0339,0.0910,14.4004,...,-14.8922,175.5426,-14.8922,175.5426,46.5565,676.3041,53.0216,706.3518,2.428971e+11,3.720839e+10
9,000002.SZ,20170428,2017-03-31,1.2320,0.4487,0.3469,0.0272,9.7588,0.0249,5.9667,...,27.2249,-84.9385,27.2249,-84.9385,-5.3196,-93.2907,-11.2538,-93.4984,1.171005e+11,NaN


In [11]:
def merge_fund(x):
    fund_df = fund_dict[x.ts_code]
    if x.is_reach_close_line:
        pledge_date = x.close_date
    else:
        pledge_date = x.start_date
    for i in reversed(range(len(fund_df))):
        fund_ser = fund_df.loc[i]
        fund_date = fund_ser['end_date']
        if fund_date >= pledge_date:
            return fund_ser

In [12]:
fund_df = pledge.apply(merge_fund, axis=1)

In [13]:
fund_df.head()

,ts_code,ann_date,end_date,current_ratio,quick_ratio,cash_ratio,inv_turn,ar_turn,ca_turn,netprofit_margin,...,q_gr_yoy,q_gr_qoq,q_sales_yoy,q_sales_qoq,q_op_yoy,q_op_qoq,q_profit_yoy,q_profit_qoq,revenue,net_profit
0,000002.SZ,20190326.0,2018-12-31,1.1543,0.4856,0.3977,0.2760,197.2095,0.2574,16.5521,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,NaN
1,000002.SZ,20190326.0,2018-12-31,1.1543,0.4856,0.3977,0.2760,197.2095,0.2574,16.5521,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,NaN
2,000002.SZ,20190821.0,2019-06-30,1.1191,0.4387,0.3268,0.1153,80.7413,0.1074,13.8427,...,21.0203,88.0024,21.0203,88.0024,31.8907,273.2963,36.9291,401.5618,1.393201e+11,1.928568e+10
3,000002.SZ,20181026.0,2018-09-30,1.1612,0.4832,0.3543,0.1759,111.1831,0.1579,12.3836,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,4.927229e+10
4,000002.SZ,20181026.0,2018-09-30,1.1612,0.4832,0.3543,0.1759,111.1831,0.1579,12.3836,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,4.927229e+10


In [14]:
fund_df = fund_df.drop(['ts_code'], axis=1)

In [15]:
pledge = pd.concat([pledge, fund_df], axis=1)

In [16]:
pledge = pledge.drop(['ann_date', 'end_date'], axis=1)

In [17]:
pledge.head()

,ts_code,start_date,release_date,pledge_price,forcast_close_line,is_reach_close_line,current_ratio,quick_ratio,cash_ratio,inv_turn,...,q_gr_yoy,q_gr_qoq,q_sales_yoy,q_sales_qoq,q_op_yoy,q_op_qoq,q_profit_yoy,q_profit_qoq,revenue,net_profit
0,000002.SZ,2018-12-25,2019-08-26,22.4751,15.73257,False,1.1543,0.4856,0.3977,0.2760,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,NaN
1,000002.SZ,2018-12-18,2019-07-22,24.3593,17.05151,False,1.1543,0.4856,0.3977,0.2760,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,NaN
2,000002.SZ,2019-06-11,NaN,26.7721,18.74047,False,1.1191,0.4387,0.3268,0.1153,...,21.0203,88.0024,21.0203,88.0024,31.8907,273.2963,36.9291,401.5618,1.393201e+11,1.928568e+10
3,000002.SZ,2018-09-26,2019-06-05,22.9654,16.07578,False,1.1612,0.4832,0.3543,0.1759,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,4.927229e+10
4,000002.SZ,2018-09-28,2019-06-05,23.0231,16.11617,False,1.1612,0.4832,0.3543,0.1759,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,4.927229e+10


In [18]:
pledge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27293 entries, 0 to 27292
Data columns (total 31 columns):
ts_code                27293 non-null object
start_date             27293 non-null datetime64[ns]
release_date           16054 non-null object
pledge_price           27293 non-null float64
forcast_close_line     27293 non-null float64
is_reach_close_line    27293 non-null bool
current_ratio          26577 non-null float64
quick_ratio            26577 non-null float64
cash_ratio             26512 non-null float64
inv_turn               26068 non-null float64
ar_turn                26272 non-null float64
ca_turn                26512 non-null float64
netprofit_margin       26718 non-null float64
grossprofit_margin     26507 non-null float64
roe                    26717 non-null float64
debt_to_assets         26720 non-null float64
op_yoy                 26720 non-null float64
ebt_yoy                26720 non-null float64
assets_yoy             26720 non-null float64
tr_yoy         

In [19]:
pledge = pledge.drop(['net_profit'], axis=1)

In [20]:
pledge_release_date = pledge['release_date']
pledge = pledge.drop(['release_date'], axis=1)

In [21]:
pledge = pledge.drop_duplicates()
pledge = pledge.dropna()

In [25]:
pledge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18559 entries, 0 to 27292
Data columns (total 30 columns):
ts_code                18559 non-null object
start_date             18559 non-null datetime64[ns]
pledge_price           18559 non-null float64
forcast_close_line     18559 non-null float64
is_reach_close_line    18559 non-null bool
current_ratio          18559 non-null float64
quick_ratio            18559 non-null float64
cash_ratio             18559 non-null float64
inv_turn               18559 non-null float64
ar_turn                18559 non-null float64
ca_turn                18559 non-null float64
netprofit_margin       18559 non-null float64
grossprofit_margin     18559 non-null float64
roe                    18559 non-null float64
debt_to_assets         18559 non-null float64
op_yoy                 18559 non-null float64
ebt_yoy                18559 non-null float64
assets_yoy             18559 non-null float64
tr_yoy                 18559 non-null float64
or_yoy        

In [23]:
pledge['release_date'] = pledge_release_date

In [24]:
pledge.head()

,ts_code,start_date,pledge_price,forcast_close_line,is_reach_close_line,current_ratio,quick_ratio,cash_ratio,inv_turn,ar_turn,...,q_gr_yoy,q_gr_qoq,q_sales_yoy,q_sales_qoq,q_op_yoy,q_op_qoq,q_profit_yoy,q_profit_qoq,revenue,release_date
0,000002.SZ,2018-12-25,22.4751,15.73257,False,1.1543,0.4856,0.3977,0.2760,197.2095,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,2019-08-26
1,000002.SZ,2018-12-18,24.3593,17.05151,False,1.1543,0.4856,0.3977,0.2760,197.2095,...,-3.2906,73.6779,-3.2906,73.6779,18.9694,198.0835,22.6986,232.0102,4.837463e+10,2019-07-22
2,000002.SZ,2019-06-11,26.7721,18.74047,False,1.1191,0.4387,0.3268,0.1153,80.7413,...,21.0203,88.0024,21.0203,88.0024,31.8907,273.2963,36.9291,401.5618,1.393201e+11,NaN
3,000002.SZ,2018-09-26,22.9654,16.07578,False,1.1612,0.4832,0.3543,0.1759,111.1831,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,2019-06-05
4,000002.SZ,2018-09-28,23.0231,16.11617,False,1.1612,0.4832,0.3543,0.1759,111.1831,...,48.1234,-6.7883,48.1234,-6.7883,71.2685,-27.8129,73.7196,-29.5315,4.837463e+10,2019-06-05


In [26]:
pledge.to_csv('../data/pledge/pledge_company_neg.csv', index=False)